# 任务 2：使用 SageMaker Debugger

在本实验中，您将使用 Amazon SageMaker Debugger 来分析和检测训练任务期间的瓶颈、资源利用率和各种训练问题，并收到相关提醒。

## 任务 2.1：环境设置

安装程序包和依赖项。

In [ ]:
%%capture

#install a compiler
%conda install -c conda-forge gcc -y

**注意**：GCC 可能需要长达 5 分钟的时间才能完成安装。

In [ ]:
#install-dependencies

%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install -U sagemaker
%pip install -U smdebug
%pip install -U shap

In [ ]:
#import libraries and set variable values

import sys
import boto3
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sagemaker
import sagemaker_datawrangler 
import shap

from mpl_toolkits.axes_grid1 import host_subplot

from sagemaker.debugger import (
    CollectionConfig,
    DebuggerHookConfig,
    FrameworkProfile,
    ProfilerConfig,
    ProfilerRule,
    Rule,
    rule_configs,
    TensorBoardOutputConfig
)

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sagemaker.xgboost.estimator import XGBoost
from smdebug.core import modes
from smdebug.trials import create_trial

base_job_name = "lab-7-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "lab-7-smdebugger"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
save_interval = 5

接下来，导入数据集。

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

将数据集拆分为训练数据集 (70%)、验证数据集 (20%) 和测试数据集 (10%)。在本实验中，您将使用训练和验证数据集来创建模型。您在本实验中不会用到测试数据集。

In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

feature_names = list(train_data.columns)

现在，将数据集上传到 Amazon Simple Storage Service (Amazon S3)。

In [ ]:
#upload-dataset
sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## 任务 2.2：修改训练脚本以启用 SageMaker Debugger

您必须修改在上个实验中使用的训练脚本，以将张量保存到指定的输出 S3 存储桶，指定要保存的张量，并注册调试挂钩。

要训练模型，您需要配置以下各项：
- **Debugger Hook Parameters**（Debugger 挂钩参数），用于在训练阶段调整输出张量的保存间隔
- **Debugger Rule Object**（Debugger 规则对象），用于保存输出张量以供评估

对于 **Debugger Hook Parameters**（Debugger 挂钩参数），您需要配置要在训练期间捕获的 **metrics**、**feature_importance**、**full_shap** 和 **average_shap** 内置张量集合。它们是在 **collection_configs** 中为 **debugger_hook_config** 配置的。**full_shap** 和 **average_shap** 内置张量集合使用夏普里值 (SHAP)。SHAP 通过以下方式来解释机器学习 (ML) 预测：假设训练数据实例的每个特征值都是某游戏（其中的预测是支出）中的一个玩家。然后，它将指示如何在各个特征之间公平地分配支出。有关 SHAP 的更多信息，请参阅 [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)。

对于 **Debugger Rule Object**（Debugger 规则对象），请在 **rules**（规则）中配置以下 **rule_configs**：
- [Profiler 报告](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report.html#debugger-profiling-report)：运行系统瓶颈检测规则并自动生成分析报告。
- [XGBoost 训练报告](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report-xgboost.html)：运行全面的 XGBoost 报告。
- [过度拟合](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overfit)：通过比较验证损失和训练损失来检测模型是否被过度拟合到训练数据。
- [过度训练](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overtraining)：检测模型是否被过度训练。对表现良好的模型进行多次训练迭代（训练和验证损失都减少）后，模型将接近损失函数的最小值，无法进一步改进。如果模型继续训练，验证损失可能会开始增加，因为模型开始过度拟合。
- [损失未减少](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#loss-not-decreasing)：检测损失的值何时没有以适当的速度减少。这些损失必须是标量。

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type='ml.m5.4xlarge',
    #Set the hyperparameters
    hyperparameters= {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "binary:logistic",
        "num_round": "300",
    },
    sagemaker_session=sagemaker_session,
    max_run=1800,
    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[  # For each of these, a new processing job will be run later in the lab
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
    ),
    #Set the Debugger Profiler Configuration
    profiler_config = ProfilerConfig(
        system_monitor_interval_millis=500,
        framework_profile_params=FrameworkProfile()

    ),
    #Configure the Debugger Rule Object
    rules = [
        ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
        Rule.sagemaker(rule_configs.create_xgboost_report()),  
        Rule.sagemaker(rule_configs.overfit()),
        Rule.sagemaker(rule_configs.overtraining()),
        Rule.sagemaker(rule_configs.loss_not_decreasing(),
            rule_parameters={
                "collection_names": "metrics",
                "num_steps": str(save_interval * 2),
            }
        )
    ]
)

## 任务 2.3：运行启用 Debugger 的训练任务

现在，使用启用 Debugger 的脚本训练 XGBoost 模型。运行训练大约需要 5–10 分钟。您可以在训练任务运行期间继续执行下一个任务，并使用 SageMaker Debugger 监控任务进度。

In [ ]:
#train-model
xgb.fit(
    inputs = data_inputs
)

## 任务 2.4：监控训练任务状态

在 SageMaker Studio 中，您可以查看试用组件，包括您启动的所有 SageMaker Debugger 任务。在本实验中，您创建了一个 **XGBoost 报告**任务、一个**过度拟合**任务、一个**过度训练**任务和一个**损失未减少**任务。在 SageMaker Studio 中浏览这些任务。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_7.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_7.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览特征组时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。

1.选择 **SageMaker Home**（SageMaker 主页）图标。
2.选择 **Experiments**（实验）。

此时 SageMaker Studio 将打开 **Experiments**（实验）选项卡。

3.选择 **Unassigned runs**（未分配的运行）。
4.从列表中选择类型为 **SageMakerTrainingJob** 的任务的 **Name**（名称）。

此时系统会显示相应训练任务的详细信息。

5.选择左侧的 **Debugger** 选项卡。

SageMaker Debugger 提供了训练任务的状态，您可以在模型训练运行期间对其进行监控。完成后，您将看到任何指定训练问题的状态。

当所有 **Status**（状态）行都显示 **No Issues Found**（未发现问题）或 **Issues Found**（发现了问题）时，分析就完成了。Debugger 规则可能需要长达 9 分钟的时间才能完成。

如果发现了问题，则意味着您的模型中可能有需要解决的问题。这些任务是否存在问题？ 

在本实验中，您不用解决发现的任何问题。但是，如果您想解决发现的问题，您可以处理数据集并使用经过调整的超参数重新训练模型来解决这些问题。

6.分析完成后，返回标记为 **lab_7.ipynb** 的笔记本选项卡。

## 任务 2.5：执行训练后分析

借助 SageMaker Debugger，您可以在 Amazon CloudWatch 中创建可用于配置自定义警报的处理任务日志。在这里，您可以打印为每个评估的指标存储日志的位置。

In [ ]:
#print-urls
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
    """Helper function to get the rule job name with correct casing"""
    return "{}-{}-{}".format(
        training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
    )


def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(
        region, region, rule_job_name
    )


def get_rule_jobs_cw_urls(xgb):
    region = boto3.Session().region_name
    training_job = xgb.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]

    result = {}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(
                training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"]
            )
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(
                rule_job_name, region
            )
    return result


get_rule_jobs_cw_urls(xgb)

除了来自张量的自定义集合外，还可以通过 SageMaker Debugger 从您的训练任务创建的默认集合（如权重、梯度、偏差和损失）检索张量。为保存的张量生成名称和值的列表，以确定要绘制哪些张量以供进一步分析。

In [ ]:
#retrieve-names
trial = create_trial(xgb.latest_job_debugger_artifacts_path())
trial.tensor_names()

In [ ]:
#retrieve-values
trial.tensor("average_shap/f1").values()

In [ ]:
#plot-tensors
shap_values = trial.tensor("full_shap/f10").value(trial.last_complete_step)
shap_no_base = shap_values[:, :-1]
shap_base_value = shap_values[0, -1]
shap.summary_plot(shap_no_base, plot_type="bar", feature_names=feature_names)

## 任务 2.6：访问 SageMaker Debugger 见解控制面板

1.返回 **Experiments**（实验）选项卡。
2.选择左侧的 **Debugger**。
3.在 **Debugger insights**（Debugger 见解）部分，选择 **Training job name**（训练任务名称）列表中的可用 **lab-7-smdebugger-job**。

此时 SageMaker Studio 将为该任务打开一个新的 **Debugger insights**（Debugger 见解）选项卡，然后开始加载数据。

SageMaker Debugger 概述了您在 Amazon Elastic Compute Cloud (Amazon EC2) 实例上的模型训练性能。在 SageMaker Studio 中浏览 SageMaker Debugger，然后检查报告中包含的详细信息。

**Systems Metrics**（系统指标）选项卡包含以下部分：

- **Resource utilization summary**（资源利用率摘要）
- **CPU utilization summary**（CPU 利用率摘要）
- **GPU Utilization summary**（GPU 利用率摘要）

**Rules**（规则）选项卡包含以下 **Insights**（见解）： 

- **BatchSize**
- **LowGPUUtiliztion**
- **CPUBottleneck**
- **GPUMemoryIncrease**
- **StepOutlier**
- **MaxInitializationTime**
- **IOBottleneck**
- **LoadBalancing**

如果发现了任何问题，数据将填充在图表和表格中。

4.您可以通过选择 **Debugger insights**（Debugger 见解）选项卡顶部附近的 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Download report</span>（下载报告）来下载 Debugger 报告。

### 总结

恭喜！ 您已使用 SageMaker Debugger 来分析和检测模型训练中的潜在问题并针对这些问题创建相关提醒。现在，您可以利用从报告和提醒生成的信息来深入了解如何高效地训练和创建更有效的模型。下一个实验的重点是使用 SageMaker Clarify 检测偏差并为模型预测提供可解释性。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。